```
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
pip install transformers datasets
pip install trl
```

In [1]:
import transformers
import torch

# モデルとトークナイザの準備
model = transformers.AutoModelForCausalLM.from_pretrained(
    "cyberagent/open-calm-small",
    torch_dtype=torch.bfloat16
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "cyberagent/open-calm-small"
)

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [2]:
print(torch.cuda.is_available())

True


In [3]:
import datasets

# データセットを用意
datadic = datasets.load_dataset("kunishou/databricks-dolly-15k-ja")
dataset = datadic['train']
dataset

Dataset({
    features: ['output', 'input', 'index', 'category', 'instruction'],
    num_rows: 15015
})

In [4]:
dataset = dataset.filter(lambda data: data['input'] == "")  # 対象を絞る

if len(dataset) > 5000:
    dataset = dataset.take(5000)  # データ量を減らす

In [6]:
args=transformers.TrainingArguments(
        output_dir="./output",
        per_device_train_batch_size=2,  # バッチサイズを小さく設定
        gradient_accumulation_steps=8,  # 勾配を蓄積するステップ数を大きく設定
        max_steps=1000,  # 最大ステップ数を設定
        logging_steps=100,
        save_steps=500,
        learning_rate=1e-5,
        weight_decay=0.01,
    )

In [7]:
# プロンプトフォーマットを準備
def prompt_format(example):
    output = []
    for i in range(len(example['instruction'])):
        text = (
            "以下はタスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n"
            f"### 指示:\n{example['instruction'][i]}\n"
            f"### 応答:\n{example['output'][i]}\n"
            + tokenizer.eos_token
        )
        output.append(text)
    return output

response_template = "### 応答:\n"

import trl

# トレイナの準備
trainer = trl.SFTTrainer(
    model=model,
    data_collator=trl.DataCollatorForCompletionOnlyLM(
        response_template,
        tokenizer=tokenizer
    ),
    args=args,
    train_dataset=dataset,
    formatting_func=prompt_format
)

In [8]:
trainer.dataset_batch_size

1000

In [9]:
# トレーニングする
trainer.train()

Step,Training Loss
100,27.487700
200,27.351500
300,26.989000
400,26.707000
500,26.703900
600,26.858400
700,26.417100
800,26.761300
900,26.745200
1000,26.629500


TrainOutput(global_step=1000, training_loss=26.86504296875, metrics={'train_runtime': 940.4181, 'train_samples_per_second': 17.014, 'train_steps_per_second': 1.063, 'total_flos': 1938109568937984.0, 'train_loss': 26.86504296875, 'epoch': 3.1952})

In [10]:
trainer.save_model("./trained_model")